# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
import sqlite3
print("sqlite3 version:", sqlite3.sqlite_version)

sqlite3 version: 3.51.0


In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [ ]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [3]:
# TODO: Load environment variables
load_dotenv('.env')
assert os.getenv('OPENAI_API_KEY') is not None
assert os.getenv('TAVILY_API_KEY') is not None

### VectorDB Instance

In [4]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
# chroma_client = chromadb.PersistentClient(path="chromadb")
import chromadb

# Create a persistent Chroma database stored locally in a folder called "chromadb"
chroma_client = chromadb.PersistentClient(path="./chromadb")

print("ChromaDB client created successfully!")

ChromaDB client created successfully!


### Collection

In [5]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
# embedding_fn = embedding_functions.OpenAIEmbeddingFunction()
from chromadb.utils import embedding_functions
import os

embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)

In [6]:
# TODO: Create a collection
# Choose any name you want
# collection = chroma_client.create_collection(
#    name="udaplay",
#    embedding_function=embedding_fn
#)
collection = chroma_client.get_or_create_collection(
    name="game_docs",
    embedding_function=embedding_fn
)

### Add documents

In [7]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [8]:
query_text = "realistic racing game with cars and tracks"

results = collection.query(
    query_texts=[query_text],
    n_results=3
)

print("Query:", query_text)
print("Top Results:")
for doc in results["documents"][0]:
    print("-", doc[:200], "...")

Query: realistic racing game with cars and tracks
Top Results:
- [PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre. ...
- [PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics. ...
- [PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson. ...
